In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression

# sns.set(rc={'figure.figsize':(11.7,8.27)})

import gzip

import import_ipynb
from My_Functions import null_cols, rmse

importing Jupyter notebook from My_Functions.ipynb


In [2]:
acc = pd.read_csv('../../accepted_2007_to_2018Q4.csv.gz',
                  compression='gzip',
                  header=0,
                  sep=',',
                  quotechar='"')
acc

C:\Users\tdcho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

In [3]:
# we will remove those rows
rows_to_drop_id = list(acc[acc['id'].str.contains('Total|Loans', na=False)].index)

acc = acc.drop(rows_to_drop_id)
acc

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

# Data Cleaning

In [4]:
# drop unneeded columns

acc_clean = acc.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                      'next_pymnt_d', 'out_prncp', 'out_prncp_inv', 'pymnt_plan', 'recoveries',
                      'title', 'total_pymnt', 'total_pymnt_inv','total_rec_int', 'total_rec_late_fee',
                      'total_rec_prncp', 'url', 'hardship_flag', 'hardship_type', 'hardship_reason',
                      'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date',
                      'hardship_end_date', 'payment_plan_start_date', 'hardship_length',
                      'hardship_dpd', 'hardship_loan_status',
                      'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 
                      'hardship_last_payment_amount', 'disbursement_method', 'debt_settlement_flag', 
                      'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
                      'settlement_amount',  'settlement_percentage', 'settlement_term',
                      'last_pymnt_d', 'last_pymnt_amnt', 'last_fico_range_high',
                      'last_fico_range_low'], axis=1)

acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,home_improvement,605xx,IL,10.78,0.0,Aug-2000,695.0,699.0,0.0,NaN,NaN,6.0,0.0,7869.0,56.2,18.0,w,Mar-2019,0.0,NaN,1.0,Joint App,71000.0,13.85,Not Verified,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,NaN,10.0,NaN,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,debt_consolidation,076xx,NJ,17.06,0.0,Sep-2008,785.0,789.0,0.0,NaN,NaN,13.0,0.0,7802.0,11.6,17.0,w,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,301500.0,1.0,1.0,0.0,1.0,23.0,12609.0,70.0,1.0,1.0,6987.0,45.0,67300.0,0.0,1.0,0.0,2.0,23192.0,54962.0,12.1,0.0,0.0,36.0,87.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,0.0,4.0,5.0,8.0,10.0,2.0,10.0,13.0,5.0,13.0,0.0,0.0,0.0,1.0,100.0,0.0

## Employment Length

In [5]:
acc_clean['emp_length'].unique()

array(['10+ years', '3 years', '4 years', '6 years', '1 year', '7 years',
       '8 years', '5 years', '2 years', '9 years', '< 1 year', nan],
      dtype=object)

In [6]:
# if employment length is 10+ years, insert 10 years

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='10+ years',
                                   '10 years', acc_clean['emp_length'])


# if employment length is 1 year , insert 1 years

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='1 year',
                                   '1 years', acc_clean['emp_length'])


# if employment length is <1, insert 0

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='< 1 year',
                                   '0 years', acc_clean['emp_length'])


# if employment length is null, insert -1

acc_clean['emp_length'] = np.where(acc_clean['emp_length'].isnull(),
                                   '-1 years', acc_clean['emp_length'])

acc_clean['emp_length']

0          10 years
1          10 years
2          10 years
3          10 years
4           3 years
             ...   
2260694     7 years
2260695    10 years
2260696     9 years
2260697     6 years
2260698    10 years
Name: emp_length, Length: 2260668, dtype: object

In [7]:
# removing ' years' from the column

acc_clean['emp_length'] = acc_clean['emp_length'].str.replace(' years', '', regex=False)


# Convert to numeric type

acc_clean['emp_length'] = pd.to_numeric(acc_clean['emp_length'])

acc_clean['emp_length'].unique()

array([10,  3,  4,  6,  1,  7,  8,  5,  2,  9,  0, -1], dtype=int64)

## Zip Code

In [8]:
# remove string values and convert to numeric

acc_clean['zip_code'] = acc_clean['zip_code'].str.extract(r'(\d+)', expand=False)
acc_clean['zip_code'] = acc_clean['zip_code'].astype(str)
acc_clean['zip_code']

0          190
1          577
2          605
3          076
4          174
          ... 
2260694    356
2260695    840
2260696    907
2260697    334
2260698    770
Name: zip_code, Length: 2260668, dtype: object

In [9]:
acc_clean['zip_code'].value_counts()

112    23908
945    23782
750    23649
606    21192
300    20497
       ...  
938        1
515        1
513        1
649        1
514        1
Name: zip_code, Length: 957, dtype: int64

# Targeted Analysis

In [10]:
acc['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [11]:
acc['debt_settlement_flag'].value_counts()

N    2226422
Y      34246
Name: debt_settlement_flag, dtype: int64

In [12]:
acc['hardship_flag'].value_counts()

N    2259836
Y        832
Name: hardship_flag, dtype: int64

In [13]:
# Dataframe of Charged Off or Defaulted or have either Debt Settlement or Hardship Flags

acc[(acc['loan_status'].str.contains('Charged|Default', na=False)) |\
    (acc['debt_settlement_flag']=='Y') | (acc['hardship_flag']=='Y')]\
[['loan_status', 'debt_settlement_flag', 'hardship_flag']].value_counts()

loan_status                                          debt_settlement_flag  hardship_flag
Charged Off                                          N                     N                235288
                                                     Y                     N                 33271
Late (31-120 days)                                   Y                     N                   905
Does not meet the credit policy. Status:Charged Off  N                     N                   751
Late (31-120 days)                                   N                     Y                   729
Late (16-30 days)                                    N                     Y                    83
Default                                              N                     N                    34
Current                                              Y                     N                    28
In Grace Period                                      N                     Y                    16
Late (16-30 days)   

In [14]:
# Dataframe of loans that are Charged Off or Defaulted

acc[(acc['loan_status'].str.contains('Charged|Default', na=False))]\
[['loan_status']].value_counts()

loan_status                                        
Charged Off                                            268559
Does not meet the credit policy. Status:Charged Off       761
Default                                                    40
dtype: int64

In [15]:
# Dataframe of Charged Off or Defaulted with that have either Debt Settlement or Hardship Flags

acc[(acc['loan_status'].str.contains('Charged|Default', na=False)) &\
    ((acc['debt_settlement_flag']=='Y') | (acc['hardship_flag']=='Y'))]\
[['loan_status', 'debt_settlement_flag', 'hardship_flag']].value_counts()

loan_status                                          debt_settlement_flag  hardship_flag
Charged Off                                          Y                     N                33271
Does not meet the credit policy. Status:Charged Off  Y                     N                   10
Default                                              Y                     N                    6
dtype: int64

In [16]:
# there are no loans that have been charged off or defaulted and have hardship flags

acc[(acc['loan_status'].str.contains('Charged|Default', na=False)) & (acc['hardship_flag']=='Y')]\
[['loan_status', 'debt_settlement_flag', 'hardship_flag']].value_counts()

Series([], dtype: int64)

In [17]:
# loans that are current and have hardship flags

acc[(acc['loan_status'].str.contains('Current', na=False)) & (acc['hardship_flag']=='Y')]\
[['loan_status', 'debt_settlement_flag', 'hardship_flag']].value_counts()

loan_status  debt_settlement_flag  hardship_flag
Current      N                     Y                4
dtype: int64

In [18]:
# Breakdown of loans that are fully paid, late, or in grace period,
# but have either a debt_settlement flag or hardship_flag

acc[(acc['loan_status'].str.contains('Paid|Late|Grace', na=False)) &\
    ((acc['debt_settlement_flag']=='Y') | (acc['hardship_flag']=='Y'))]\
[['loan_status', 'debt_settlement_flag', 'hardship_flag']].value_counts()

loan_status         debt_settlement_flag  hardship_flag
Late (31-120 days)  Y                     N                905
                    N                     Y                729
Late (16-30 days)   N                     Y                 83
In Grace Period     N                     Y                 16
Late (16-30 days)   Y                     N                 11
In Grace Period     Y                     N                 10
Fully Paid          Y                     N                  5
dtype: int64

## Breaking up the dataframes after cleaning and analysis

In [19]:
# keeping only the rows that have loan status of Paid Off, Default, or Charged Off

acc_for_final = acc_clean[(acc_clean['loan_status'].str.contains('Paid|Default|Charged', na=False))]
acc_for_final = acc_for_final.reset_index()
acc_for_final = acc_for_final.drop('index', axis=1)
acc_for_final

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,190,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,small_business,577,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,home_improvement,605,IL,10.78,0.0,Aug-2000,695.0,699.0,0.0,NaN,NaN,6.0,0.0,7869.0,56.2,18.0,w,Mar-2019,0.0,NaN,1.0,Joint App,71000.0,13.85,Not Verified,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,NaN,10.0,NaN,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,major_purchase,174,PA,25.37,1.0,Jun-1998,695.0,699.0,3.0,12.0,NaN,12.0,0.0,21929.0,64.5,35.0,w,Mar-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,331730.0,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0

In [20]:
# create target label

target_final = acc_for_final[['loan_status']]

In [21]:
target_final[['loan_status']].value_counts()

loan_status                                        
Fully Paid                                             1076751
Charged Off                                             268559
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
dtype: int64

In [22]:
# converting loan statuses to binary (0, 1)

target_final['loan_status_binary'] = 1

# create mask for clarity on what is being changed
mask_fully = target_final['loan_status'].str.contains('Fully', na=False)

target_final.loc[mask_fully, 'loan_status_binary'] = 0

target_final[['loan_status_binary']].value_counts()

<ipython-input-22-153f0427a3d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_final['loan_status_binary'] = 1
C:\Users\tdcho\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


loan_status_binary
0                     1078739
1                      269360
dtype: int64

In [23]:
# dropping target label

acc_for_final = acc_for_final.drop(['loan_status'], axis=1)

acc_for_final

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10,MORTGAGE,55000.0,Not Verified,Dec-2015,debt_consolidation,190,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10,MORTGAGE,65000.0,Not Verified,Dec-2015,small_business,577,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10,MORTGAGE,63000.0,Not Verified,Dec-2015,home_improvement,605,IL,10.78,0.0,Aug-2000,695.0,699.0,0.0,NaN,NaN,6.0,0.0,7869.0,56.2,18.0,w,Mar-2019,0.0,NaN,1.0,Joint App,71000.0,13.85,Not Verified,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,NaN,10.0,NaN,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3,MORTGAGE,104433.0,Source Verified,Dec-2015,major_purchase,174,PA,25.37,1.0,Jun-1998,695.0,699.0,3.0,12.0,NaN,12.0,0.0,21929.0,64.5,35.0,w,Mar-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,331730.0,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1

## Check if columns with dtype object should be ordinal

In [24]:
# Get columns with string values

cols_w_strings = acc_for_final.loc[:, acc_for_final.dtypes == object].columns

acc_for_final[cols_w_strings]

,term,grade,sub_grade,emp_title,home_ownership,verification_status,issue_d,purpose,zip_code,addr_state,earliest_cr_line,initial_list_status,last_credit_pull_d,application_type,verification_status_joint,sec_app_earliest_cr_line
0,36 months,C,C4,leadman,MORTGAGE,Not Verified,Dec-2015,debt_consolidation,190,PA,Aug-2003,w,Mar-2019,Individual,NaN,NaN
1,36 months,C,C1,Engineer,MORTGAGE,Not Verified,Dec-2015,small_business,577,SD,Dec-1999,w,Mar-2019,Individual,NaN,NaN
2,60 months,B,B4,truck driver,MORTGAGE,Not Verified,Dec-2015,home_improvement,605,IL,Aug-2000,w,Mar-2019,Joint App,Not Verified,NaN
3,60 months,F,F1,Contract Specialist,MORTGAGE,Source Verified,Dec-2015,major_purchase,174,PA,Jun-1998,w,Mar-2018,Individual,NaN,NaN
4,36 months,C,C3,Veterinary Tecnician,RENT,Source Verified,Dec-2015,debt_consolidation,300,GA,Oct-1987,w,May-2017,Individual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348094,60 months,B,B2,NaN,OWN,Not Verified,Oct-2016,home_improvement,775,TX,Jul-2004,f,Mar-2019,Individual,NaN,NaN
1348095,60 months,C,C3,Chief Operating Officer,MORTGAGE,Not Verified,Oct-2016,debt_consolidation,900,CA,Mar-2002,f,Jul-2018,Individual,NaN,NaN
1348096,60 months,C,C4,Sales Manager,MORTGAGE,Source Verified,Oct-2016,home_improvement,863,AZ,Jun-2011,f,Dec-2018,Individual,NaN,NaN
1348097,60 months,C,C1,Manager,RENT,Not Verified,Oct-2016,medical,284,NC,Aug-1997,f,Mar-2019,Individual,NaN,NaN


In [25]:
# getting the unique values for columns that have object dtype

for col in cols_w_strings:
    print(col)
    print(acc_for_final[col].unique(),'\n')


term
[' 36 months' ' 60 months'] 

grade
['C' 'B' 'F' 'A' 'E' 'D' 'G'] 

sub_grade
['C4' 'C1' 'B4' 'F1' 'C3' 'B2' 'B1' 'A2' 'B5' 'C2' 'E2' 'A4' 'E3' 'C5'
 'A1' 'D4' 'F3' 'D1' 'B3' 'D3' 'D5' 'A5' 'F2' 'E4' 'D2' 'E1' 'F5' 'E5'
 'A3' 'G2' 'G1' 'G3' 'G4' 'F4' 'G5'] 

emp_title
['leadman' 'Engineer' 'truck driver' ... 'personal investor'
 'Sr. Photo Producer' 'Exec Dir Internal Audit'] 

home_ownership
['MORTGAGE' 'RENT' 'OWN' 'ANY' 'NONE' 'OTHER'] 

verification_status
['Not Verified' 'Source Verified' 'Verified'] 

issue_d
['Dec-2015' 'Nov-2015' 'Oct-2015' 'Sep-2015' 'Aug-2015' 'Jul-2015'
 'Jun-2015' 'May-2015' 'Apr-2015' 'Mar-2015' 'Feb-2015' 'Jan-2015'
 'Mar-2018' 'Feb-2018' 'Jan-2018' 'Sep-2017' 'Aug-2017' 'Jul-2017'
 'Jun-2016' 'May-2016' 'Apr-2016' 'Sep-2018' 'Aug-2018' 'Jul-2018'
 'Jun-2017' 'May-2017' 'Apr-2017' 'Mar-2016' 'Feb-2016' 'Jan-2016'
 'Dec-2014' 'Nov-2014' 'Oct-2014' 'Sep-2014' 'Aug-2014' 'Jul-2014'
 'Jun-2014' 'May-2014' 'Apr-2014' 'Mar-2014' 'Feb-2014' 'Jan-2014'
 'Dec

['Mar-2019' 'Mar-2018' 'May-2017' 'Nov-2018' 'Sep-2018' 'Nov-2017'
 'Aug-2018' 'Apr-2017' 'Jun-2016' 'Jan-2018' 'Sep-2017' 'Dec-2017'
 'Jul-2016' 'Jan-2019' 'Jun-2017' 'Feb-2019' 'Dec-2018' 'Feb-2018'
 'Aug-2017' 'Feb-2017' 'May-2018' 'Dec-2016' 'Apr-2018' 'Jul-2018'
 'Mar-2016' 'Jun-2018' 'Oct-2017' 'Jul-2017' 'Oct-2018' 'Jan-2017'
 'Oct-2016' 'Apr-2016' 'Mar-2017' 'Aug-2016' 'Feb-2016' 'Nov-2016'
 'Sep-2016' 'Jan-2016' 'Dec-2015' 'May-2016' nan 'Nov-2015' 'Oct-2015'
 'Sep-2015' 'Aug-2015' 'Jul-2015' 'Jun-2015' 'May-2015' 'Apr-2015'
 'Mar-2015' 'Feb-2015' 'Jan-2015' 'Dec-2014' 'Nov-2014' 'Oct-2014'
 'Sep-2014' 'Aug-2014' 'Jul-2014' 'Jun-2014' 'May-2014' 'Apr-2014'
 'Mar-2014' 'Feb-2014' 'Jan-2014' 'Sep-2012' 'Jun-2012' 'Dec-2012'
 'Feb-2013' 'Feb-2012' 'Nov-2013' 'Apr-2012' 'May-2013' 'Aug-2012'
 'Sep-2013' 'Jan-2012' 'Oct-2013' 'May-2012' 'Jul-2012' 'Jun-2013'
 'Dec-2013' 'Nov-2012' 'Mar-2013' 'Apr-2013' 'Jul-2013' 'Mar-2012'
 'Aug-2013' 'Oct-2012' 'Nov-2011' 'Dec-2011' 'Oct-2011' 'S

# Grade

In [26]:
# Grouping E, F, and G into F

acc_for_final['grouped_grade_NO_EG'] = acc_for_final['grade']

acc_for_final['grouped_grade_NO_EG'] = acc_for_final['grouped_grade_NO_EG'].str.replace('E',
                                                                                        'F',
                                                                                        regex=False)

acc_for_final['grouped_grade_NO_EG'] = acc_for_final['grouped_grade_NO_EG'].str.replace('G',
                                                                                        'F',
                                                                                        regex=False)

In [27]:
# replacing letter grades with numbers

# get sorted list of grades
grade_list = sorted(list(acc_for_final['grade'].unique()))

i = 1

for grades in grade_list:
    acc_for_final.loc[acc_for_final['grade']==grades, 'grade'] = i
    i+=1

acc_for_final['grade'] = acc_for_final['grade'].astype(int)
acc_for_final['grade'].unique()

array([3, 2, 6, 1, 5, 4, 7])

In [28]:
# replacing letter grades with numbers for grouped grades

# get sorted list of grades
grade_list = sorted(list(acc_for_final['grouped_grade_NO_EG'].unique()))

i = 1

for grades in grade_list:
    acc_for_final.loc[acc_for_final['grouped_grade_NO_EG']==grades, 'grouped_grade_NO_EG'] = i
    i+=1

acc_for_final['grouped_grade_NO_EG'] = acc_for_final['grouped_grade_NO_EG'].astype(int)
acc_for_final['grouped_grade_NO_EG'].unique()

array([3, 2, 5, 1, 4])

### Subgrade

# START HERE

In [29]:
# grouping subgrades 1, 2 vs 3, 4, 5

acc_for_final['grouped_subgrade_12_345'] = acc_for_final['sub_grade']

acc_for_final['grouped_subgrade_12_345'] = acc_for_final['grouped_subgrade_12_345'].str.replace('2',
                                                                                  '1', regex=False)

acc_for_final['grouped_subgrade_12_345'] = acc_for_final['grouped_subgrade_12_345'].str.replace('3',
                                                                                  '2', regex=False)

acc_for_final['grouped_subgrade_12_345'] = acc_for_final['grouped_subgrade_12_345'].str.replace('4',
                                                                                  '2', regex=False)

acc_for_final['grouped_subgrade_12_345'] = acc_for_final['grouped_subgrade_12_345'].str.replace('5',
                                                                                  '2', regex=False)

In [30]:
acc_for_final['grouped_subgrade_12_345'].value_counts()

B2    247816
C2    217347
C1    164976
A2    154321
B1    145286
D2    105228
D1     96429
A1     80872
E2     48817
E1     45375
F1     17288
F2     15018
G1      5193
G2      4133
Name: grouped_subgrade_12_345, dtype: int64

In [31]:
# grouping subgrades 1, 2, 3 vs 4, 5

acc_for_final['grouped_subgrade_123_45'] = acc_for_final['sub_grade']

acc_for_final['grouped_subgrade_123_45'] = acc_for_final['grouped_subgrade_123_45'].str.replace('2',
                                                                                  '1', regex=False)

acc_for_final['grouped_subgrade_123_45'] = acc_for_final['grouped_subgrade_123_45'].str.replace('3',
                                                                                  '1', regex=False)

acc_for_final['grouped_subgrade_123_45'] = acc_for_final['grouped_subgrade_123_45'].str.replace('4',
                                                                                  '2', regex=False)

acc_for_final['grouped_subgrade_123_45'] = acc_for_final['grouped_subgrade_123_45'].str.replace('5',
                                                                                  '2', regex=False)

In [32]:
# grouping Subgrades

acc_for_final['grouped_subgrade'] = acc_for_final['sub_grade']

acc_for_final['grouped_subgrade'] = acc_for_final['grouped_subgrade'].str.replace('E',
                                                                                  'F', regex=False)

acc_for_final['grouped_subgrade'] = acc_for_final['grouped_subgrade'].str.replace('G',
                                                                                  'F', regex=False)

In [33]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_for_final['sub_grade'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_for_final.loc[acc_for_final['sub_grade']==subgrades, 'sub_grade'] = i
    i+=1

acc_for_final['sub_grade'] = acc_for_final['sub_grade'].astype(int)
acc_for_final['sub_grade'].unique()

array([14, 11,  9, 26, 13,  7,  6,  2, 10, 12, 22,  4, 23, 15,  1, 19, 28,
       16,  8, 18, 20,  5, 27, 24, 17, 21, 30, 25,  3, 32, 31, 33, 34, 29,
       35])

In [34]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_for_final['grouped_subgrade_12_345'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_for_final.loc[acc_for_final['grouped_subgrade_12_345']==subgrades, 'grouped_subgrade_12_345'] = i
    i+=1

acc_for_final['grouped_subgrade_12_345'] = acc_for_final['grouped_subgrade_12_345'].astype(int)
acc_for_final['grouped_subgrade_12_345'].unique()

array([ 6,  5,  4, 11,  3,  1,  9,  2, 10,  8, 12,  7, 13, 14])

In [35]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_for_final['grouped_subgrade_123_45'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_for_final.loc[acc_for_final['grouped_subgrade_123_45']==subgrades, 'grouped_subgrade_123_45'] = i
    i+=1

acc_for_final['grouped_subgrade_123_45'] = acc_for_final['grouped_subgrade_123_45'].astype(int)
acc_for_final['grouped_subgrade_123_45'].unique()

array([ 6,  5,  4, 11,  3,  1,  9,  2,  8,  7, 10, 12, 13, 14])

In [36]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_for_final['grouped_subgrade'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_for_final.loc[acc_for_final['grouped_subgrade']==subgrades, 'grouped_subgrade'] = i
    i+=1

acc_for_final['grouped_subgrade'] = acc_for_final['grouped_subgrade'].astype(int)
acc_for_final['grouped_subgrade'].unique()

array([14, 11,  9, 21, 13,  7,  6,  2, 10, 12, 22,  4, 23, 15,  1, 19, 16,
        8, 18, 20,  5, 24, 17, 25,  3])

### Zip Code

In [37]:
# dropping zip code as it will create too many new columns once dummified/one hot encoded

acc_for_final = acc_for_final.drop(['zip_code'], axis=1)

### Earliest Credit Line

In [38]:
# converting the column to datetime format

acc_for_final['earliest_cr_line_dt'] = pd.DatetimeIndex(acc_for_final['earliest_cr_line'])
acc_for_final['earliest_cr_line_dt']

0         2003-08-01
1         1999-12-01
2         2000-08-01
3         1998-06-01
4         1987-10-01
             ...    
1348094   2004-07-01
1348095   2002-03-01
1348096   2011-06-01
1348097   1997-08-01
1348098   1999-07-01
Name: earliest_cr_line_dt, Length: 1348099, dtype: datetime64[ns]

In [39]:
# creating a new column that will show the number of months that credit has existed for customer

acc_for_final['duration_of_credit_months'] =  (pd.DatetimeIndex(acc_for_final['issue_d']).year -\
                                               pd.DatetimeIndex(acc_for_final['earliest_cr_line']).year)*12 +\
(pd.DatetimeIndex(acc_for_final['issue_d']).month - pd.DatetimeIndex(acc_for_final['earliest_cr_line']).month)

acc_for_final[['duration_of_credit_months']]

,duration_of_credit_months
0,148.0
1,192.0
2,184.0
3,210.0
4,338.0
...,...
1348094,147.0
1348095,175.0
1348096,64.0
1348097,230.0


In [40]:
# dropping the columns that helped create the new column

acc_for_final = acc_for_final.drop(['earliest_cr_line_dt', 'earliest_cr_line'], axis=1)

### Last Credit Pulled Date

In [41]:
# dropping the column bc it is used post-funding

acc_for_final = acc_for_final.drop(['last_credit_pull_d'], axis=1)

### Verification status

In [42]:
acc_for_final['verification_status'].unique()

array(['Not Verified', 'Source Verified', 'Verified'], dtype=object)

In [43]:
# if the income is verified, change to 1. If not verified, change to 0

acc_for_final.loc[((acc_for_final['verification_status']=='Source Verified') |\
                  (acc_for_final['verification_status']=='Verified')), 'verification_status'] = 1

acc_for_final.loc[acc_for_final['verification_status']=='Not Verified', 'verification_status'] = 0

# change dtype to int
acc_for_final['verification_status'] = acc_for_final['verification_status'].astype(int)

acc_for_final['verification_status']

0          0
1          0
2          0
3          1
4          1
          ..
1348094    0
1348095    0
1348096    1
1348097    0
1348098    0
Name: verification_status, Length: 1348099, dtype: int32

In [44]:
acc_for_final['application_type'].value_counts()

Individual    1322293
Joint App       25806
Name: application_type, dtype: int64

In [45]:
list_sec_app = list(acc_for_final.filter(like='sec_app').columns)
list_sec_app.append('annual_inc_joint')
list_sec_app

['sec_app_fico_range_low',
 'sec_app_fico_range_high',
 'sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
 'sec_app_mths_since_last_major_derog',
 'annual_inc_joint']

In [46]:
acc_for_final[(acc_for_final['sec_app_fico_range_low'].notnull())|
              (acc_for_final['sec_app_fico_range_high'].notnull())|
              (acc_for_final['sec_app_earliest_cr_line'].notnull())|
              (acc_for_final['sec_app_inq_last_6mths'].notnull())|
              (acc_for_final['sec_app_mort_acc'].notnull())|
              (acc_for_final['sec_app_open_acc'].notnull())|
              (acc_for_final['sec_app_revol_util'].notnull())|
              (acc_for_final['sec_app_open_act_il'].notnull())|
              (acc_for_final['sec_app_num_rev_accts'].notnull())|
              (acc_for_final['sec_app_chargeoff_within_12_mths'].notnull())|
              (acc_for_final['sec_app_collections_12_mths_ex_med'].notnull())|
              (acc_for_final['sec_app_mths_since_last_major_derog'].notnull())|
              (acc_for_final['annual_inc_joint'].notnull())]


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,2,9,truck driver,10,MORTGAGE,63000.00,0,Dec-2015,home_improvement,IL,10.78,0.0,695.0,699.0,0.0,NaN,NaN,6.0,0.0,7869.0,56.2,18.0,w,0.0,NaN,1.0,Joint App,71000.00,13.85,Not Verified,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,NaN,10.0,NaN,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,4,9,184.0
388,15000.0,15000.0,15000.0,36 months,11.99,498.15,3,11,Lead Tech,1,MORTGAGE,65000.00,1,Dec-2015,debt_consolidation,CA,17.56,0.0,665.0,669.0,0.0,54.0,NaN,9.0,0.0,13028.0,88.3,16.0,w,0.0,NaN,1.0,Joint App,77000.00,15.40,Not Verified,0.0,0.0,231860.0,0.0,0.0,0.0,0.0,30.0,22525.0,74.0,0.0,2.0,2839.0,78.0,14800.0,3.0,1.0,2.0,2.0,25762.0,295.0,94.2,0.0,0.0,99.0,136.0,13.0,13.0,1.0,28.0,NaN,11.0,54.0,0.0,2.0,7.0,2.0,3.0,4.0,7.0,11.0,7.0,9.0,0.0,0.0,0.0,0.0,92.3,100.0,0.0,0.0,251645.0,35553.0,5100.0,30649.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,5,5,11,136.0
451,24000.0,24000.0,24000.0,60 months,23.13,678.37,6,27,Coordinator,6,MORTGAGE,43435.00,1,Dec-2015,debt_consolidation,TX,35.70,0.0,680.0,684.0,0.0,NaN,84.0,13.0,1.0,12508.0,73.1,22.0,w,0.0,NaN,1.0,Joint App,113314.00,25.44,Not Verified,0.0,156.0,182215.0,0.0,6.0,2.0,5.0,10.0,48582.0,75.0,0.0,1.0,5690.0,74.0,17100.0,1.0,2.0,1.0,6.0,14017.0,357.0,96.8,0.0,0.0,52.0,209.0,13.0,10.0,1.0,26.0,NaN,12.0,NaN,0.0,2.0,5.0,2.0,4.0,9.0,6.0,12.0,5.0,13.0,0.0,0.0,0.0,2.0,100.0,100.0,1.0,0.0,208064.0,61090.0,11000.0,61739.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,11,11,22,209.0
470,24000.0,24000.0,24000.0,60 months,13.44,551.50,3,13,Transportation Manager,1,MORTGAGE,96000.00,1,Dec-2015,credit_card,MO,22.70,0.0,675.0,679.0,1.0,55.0,63.0,13.0,1.0,19403.0,53.3,24.0,w,0.0,55.0,1.0,Joint App,117000.00,22.76,Not Verified,0.0,0.0,220879.0,0.0,4.0,1.0,3.0,10.0,43145.0,70.0,1.0,3.0,6210.0,64.0,36400.0,2.0,0.0,5.0,7.0,16991.0,6328.0,60.9,0.0,0.0,158.0,71.0,11.0,10.0,2.0,19.0,NaN,6.0,55.0,1.0,2.0,5.0,3.0,4.0,8.0,8.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,95.7,66.7,0.0,0.0,258301.0,62548.0,16200.0,61251.0,

In [47]:
acc_for_final.loc[acc_for_final['application_type']=='Joint App', 'application_type'] = 2
acc_for_final.loc[acc_for_final['application_type']=='Individual', 'application_type'] = 1

acc_for_final['application_type'] = acc_for_final['application_type'].astype(int)

In [48]:
acc_for_final['verification_status_joint'].unique()

array([nan, 'Not Verified', 'Verified', 'Source Verified'], dtype=object)

In [49]:
'''
for verification_status_joint where there are values in cols that start with
sec_app, but verification_status_joint is null
'''

# add in 'not verified' where applicable

acc_for_final.loc[((acc_for_final['verification_status_joint'].notnull()) &\
                   (acc_for_final['application_type']==1)),
                  'verification_status_joint'] = 'Not Verified'

In [50]:
acc_for_final[((acc_for_final['verification_status_joint'].isnull()) &\
                  (acc_for_final['application_type']==2))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months
375930,30000.0,30000.0,30000.0,36 months,5.31,903.31,1,1,Associate Teacher,2,MORTGAGE,20000.0,1,Mar-2018,debt_consolidation,FL,43.04,0.0,755.0,759.0,0.0,NaN,NaN,10.0,0.0,35332.0,25.1,17.0,w,0.0,NaN,1.0,2,68000.0,12.65,NaN,0.0,0.0,35332.0,0.0,0.0,0.0,1.0,23.0,0.0,NaN,0.0,1.0,17531.0,25.0,140700.0,0.0,0.0,0.0,2.0,3926.0,103868.0,25.4,0.0,0.0,136.0,204.0,16.0,16.0,0.0,16.0,NaN,23.0,NaN,0.0,3.0,3.0,8.0,9.0,2.0,10.0,15.0,3.0,10.0,0.0,0.0,0.0,0.0,100.0,12.5,0.0,0.0,140700.0,35332.0,139200.0,0.0,35332.0,745.0,749.0,Jan-2003,0.0,4.0,8.0,28.0,0.0,7.0,0.0,0.0,NaN,1,1,1,1,204.0
376156,32000.0,32000.0,32000.0,60 months,15.04,761.95,3,14,Project Management,10,MORTGAGE,120000.0,1,Mar-2018,debt_consolidation,WA,31.21,2.0,670.0,674.0,0.0,22.0,NaN,11.0,0.0,36061.0,57.3,29.0,w,2.0,25.0,1.0,2,146500.0,31.10,NaN,0.0,915.0,126501.0,3.0,4.0,2.0,6.0,2.0,90440.0,56.0,2.0,3.0,4127.0,57.0,57400.0,0.0,8.0,2.0,9.0,11500.0,10494.0,36.4,0.0,0.0,177.0,183.0,2.0,2.0,0.0,2.0,49.0,4.0,45.0,5.0,2.0,6.0,2.0,3.0,18.0,7.0,10.0,6.0,11.0,0.0,0.0,0.0,4.0,78.9,50.0,0.0,0.0,163642.0,126501.0,16500.0,106242.0,72228.0,635.0,639.0,Apr-2005,0.0,4.0,7.0,45.0,1.0,5.0,0.0,0.0,26.0,3,6,6,14,183.0
377507,14500.0,14500.0,14500.0,36 months,5.31,436.60,1,1,Software Developer,2,MORTGAGE,74000.0,1,Mar-2018,debt_consolidation,GA,9.22,0.0,740.0,744.0,2.0,NaN,NaN,5.0,0.0,16257.0,45.2,7.0,w,0.0,NaN,1.0,2,136000.0,9.46,NaN,0.0,0.0,176952.0,3.0,1.0,1.0,1.0,6.0,3486.0,48.0,2.0,2.0,14095.0,46.0,36000.0,1.0,0.0,4.0,4.0,35390.0,18092.0,45.2,0.0,0.0,79.0,261.0,3.0,3.0,1.0,10.0,NaN,0.0,NaN,0.0,2.0,3.0,2.0,2.0,2.0,3.0,4.0,3.0,5.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,200825.0,19743.0,33000.0,7200.0,38271.0,620.0,624.0,Sep-2010,1.0,1.0,13.0,74.4,2.0,11.0,0.0,0.0,NaN,1,1,1,1,261.0
377576,35000.0,35000.0,35000.0,60 months,18.45,897.37,4,17,Office Specialist III,2,MORTGAGE,65000.0,1,Mar-2018,debt_consolidation,CA,17.43,0.0,670.0,674.0,1.0,NaN,NaN,10.0,0.0,32693.0,74.5,12.0,w,0.0,NaN,1.0,2,148000.0,13.64,NaN,0.0,0.0,403920.0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,2.0,2.0,16322.0,75.0,43900.0,1.0,0.0,5.0,5.0,40392.0,5384.0,85.2,0.0,0.0,NaN,76.0,11.0,1.0,3.0,12.0,NaN,0.0,NaN,0.0,6.0,9.0,6.0,6.0,0.0,9.0,9.0,9.0,10.0,0.0,0.0,0.0,3.0,100.0,66.7,0.0,0.0,415127.0,32693.0,36400.0,0.0,34704.0,675.0,67

In [51]:
acc_for_final.loc[((acc_for_final['verification_status_joint'].isnull()) &\
                   (acc_for_final['application_type']==2)),
                  'verification_status_joint'] = 'Not Verified'

In [52]:
acc_for_final[((acc_for_final['verification_status_joint'].notnull()) &\
                  (acc_for_final['application_type']==1))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months


In [53]:
acc_for_final[((acc_for_final['verification_status_joint'].isnull()) &\
               (acc_for_final['application_type']==1))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,3,14,leadman,10,MORTGAGE,55000.0,0,Dec-2015,debt_consolidation,PA,5.91,0.0,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,30.0,1.0,1,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6,6,14,148.0
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,3,11,Engineer,10,MORTGAGE,65000.0,0,Dec-2015,small_business,SD,16.06,1.0,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,NaN,1.0,1,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,5,5,11,192.0
3,10400.0,10400.0,10400.0,60 months,22.45,289.91,6,26,Contract Specialist,3,MORTGAGE,104433.0,1,Dec-2015,major_purchase,PA,25.37,1.0,695.0,699.0,3.0,12.0,NaN,12.0,0.0,21929.0,64.5,35.0,w,0.0,NaN,1.0,1,NaN,NaN,NaN,0.0,0.0,331730.0,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,11,11,21,210.0
4,11950.0,11950.0,11950.0,36 months,13.44,405.18,3,13,Veterinary Tecnician,4,RENT,34000.0,1,Dec-2015,debt_consolidation,GA,10.20,0.0,690.0,694.0,0.0,NaN,NaN,5.0,0.0,8822.0,68.4,6.0,w,0.0,NaN,1.0,1,NaN,NaN,NaN,0.0,0.0,12798.0,0.0,1.0,0.0,0.0,338.0,3976.0,99.0,0.0,0.0,4522.0,76.0,12900.0,0.0,0.0,0.0,0.0,2560.0,844.0,91.0,0.0,0.0,338.0,54.0,32.0,32.0,0.0,36.0,NaN,NaN,NaN,0.0,2.0,3.0,2.0,2.0,2.0,4.0,4.0,3.0,5.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,16900.0,12798.0,9400.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6,5,13,338.0
5,20000.0,20000.0,20000.0,36 months,9.17,637.58,2,7,Vice Presid

In [54]:
acc_for_final.loc[((acc_for_final['verification_status_joint'].isnull()) &\
                   (acc_for_final['application_type']==1)),
                  'verification_status_joint'] = 'Individual'

In [55]:
# if the income is verified, change to 1. If not verified, change to 0

acc_for_final.loc[((acc_for_final['verification_status_joint']=='Source Verified') |\
                  (acc_for_final['verification_status_joint']=='Verified')),
                  'verification_status_joint'] = 1

acc_for_final.loc[acc_for_final['verification_status_joint']=='Individual',
                  'verification_status_joint'] = -1

acc_for_final.loc[acc_for_final['verification_status_joint']=='Not Verified',
                  'verification_status_joint'] = 0

In [56]:
acc_for_final['verification_status_joint'] = acc_for_final['verification_status_joint'].astype(int)

## Reducing Income if not Verified

In [57]:
acc_for_final['annual_inc_reduced'] = acc_for_final['annual_inc']

acc_for_final.loc[acc_for_final['verification_status']==0,
                  'annual_inc_reduced'] = acc_for_final['annual_inc'] * 0.8

In [58]:
acc_for_final['annual_inc_joint_reduced'] = acc_for_final['annual_inc_joint']

acc_for_final.loc[acc_for_final['verification_status_joint']==0,
                  'annual_inc_joint_reduced'] = acc_for_final['annual_inc_joint'] * 0.8

### Joint Earliest Credit Line

In [59]:
# converting column to datetime

acc_for_final['sec_app_earliest_cr_line_dt'] = pd.DatetimeIndex(acc_for_final['sec_app_earliest_cr_line'])
acc_for_final['sec_app_earliest_cr_line_dt']

0         NaT
1         NaT
2         NaT
3         NaT
4         NaT
           ..
1348094   NaT
1348095   NaT
1348096   NaT
1348097   NaT
1348098   NaT
Name: sec_app_earliest_cr_line_dt, Length: 1348099, dtype: datetime64[ns]

In [60]:
# checking how many non-null values there are

acc_for_final[acc_for_final['sec_app_earliest_cr_line'].notnull()]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,annual_inc_reduced,annual_inc_joint_reduced,sec_app_earliest_cr_line_dt
375562,1000.0,1000.0,1000.0,36 months,17.47,35.89,4,16,Lead Sanitation Operator,2,RENT,51000.0,1,Mar-2018,credit_card,TX,10.56,0.0,705.0,709.0,1.0,61.0,103.0,5.0,1.0,2.0,0.1,9.0,w,0.0,61.0,1.0,2,122000.0,7.47,1,0.0,0.0,22707.0,1.0,3.0,0.0,1.0,17.0,22705.0,95.0,2.0,2.0,2.0,86.0,2500.0,4.0,0.0,2.0,3.0,4541.0,2498.0,0.1,0.0,0.0,120.0,9.0,1.0,1.0,0.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,2.0,2.0,7.0,2.0,2.0,1.0,5.0,0.0,0.0,0.0,2.0,88.9,0.0,0.0,1.0,26501.0,22707.0,2500.0,24001.0,1282.0,635.0,639.0,Feb-2007,5.0,1.0,13.0,32.0,9.0,12.0,0.0,0.0,62.0,4,7,7,16,120.0,51000.0,122000.0,2007-02-01
375576,20000.0,20000.0,20000.0,60 months,30.65,655.08,6,29,Purchasing Manager,4,MORTGAGE,45000.0,1,Mar-2018,credit_card,CO,43.63,0.0,685.0,689.0,0.0,NaN,118.0,18.0,1.0,13659.0,56.9,21.0,w,0.0,NaN,1.0,2,145000.0,32.01,1,0.0,0.0,529063.0,0.0,9.0,1.0,3.0,8.0,74050.0,102.0,0.0,1.0,2412.0,85.0,24000.0,1.0,1.0,1.0,5.0,29392.0,2629.0,69.4,0.0,0.0,89.0,112.0,22.0,8.0,1.0,35.0,NaN,8.0,NaN,0.0,3.0,7.0,3.0,4.0,11.0,8.0,9.0,7.0,18.0,0.0,0.0,0.0,1.0,100.0,33.3,1.0,0.0,559142.0,87709.0,8600.0,81337.0,89861.0,670.0,674.0,Sep-2008,0.0,3.0,17.0,68.5,5.0,13.0,0.0,0.0,NaN,5,12,12,24,112.0,45000.0,145000.0,2008-09-01
375581,6000.0,6000.0,6000.0,36 months,20.39,224.18,4,19,PHLOBOTOMY,2,RENT,31000.0,0,Mar-2018,home_improvement,TX,20.79,0.0,705.0,709.0,0.0,NaN,NaN,9.0,0.0,0.0,0.0,9.0,w,0.0,NaN,1.0,2,114000.0,21.61,0,0.0,0.0,39627.0,0.0,7.0,1.0,3.0,8.0,39627.0,97.0,0.0,1.0,0.0,93.0,1500.0,1.0,0.0,3.0,4.0,4403.0,1000.0,0.0,0.0,0.0,38.0,70.0,19.0,8.0,0.0,19.0,NaN,8.0,NaN,0.0,0.0,0.0,1.0,1.0,7.0,2.0,2.0,0.0,9.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,42531.0,39627.0,1000.0,41031.0,5048.0,630.0,634.0,Aug-2006,3.0,0.0,9.0,82.8,4.0,5.0,0.0,0.0,NaN,4,8,8,19,70.0,24800.0,91200.0,2006-08-01
375584,7500.0,7500.0,7475.0,36 months,7.34,232.75,1,4,"Vice President, Client Services",10,OWN,300000.0,1,Mar-2018,other,CO,8.91,0.0,720.0,724.0,1.0,43.0,NaN,9.0,0.0,12750.0,34.6,23.0,f,0.0,68.0,1.0,2,500000.0,7.66,1,0.0,0.0,423667.0,2.0,2.0,1.0,1.0,6.0,54103.0,86.0,2.0,2.0,5143.0,57.0,36800.0,1.0,0.0,3.0,4.0,52958.0,23050.0,35.6,0.0,0.0,186.0,155.0,1.0,1.0,3.0,1.0,54.0,5.0,54.0,1.0,3.0,3.0,5.0,7.0,11.0,6.0,9.0,3.0,9

In [61]:
# confirming that we can use 0 as a plausible value for null values

acc_for_final[acc_for_final['sec_app_earliest_cr_line']<=acc_for_final['issue_d']]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,annual_inc_reduced,annual_inc_joint_reduced,sec_app_earliest_cr_line_dt
375562,1000.0,1000.0,1000.0,36 months,17.47,35.89,4,16,Lead Sanitation Operator,2,RENT,51000.0,1,Mar-2018,credit_card,TX,10.56,0.0,705.0,709.0,1.0,61.0,103.0,5.0,1.0,2.0,0.1,9.0,w,0.0,61.0,1.0,2,122000.0,7.47,1,0.0,0.0,22707.0,1.0,3.0,0.0,1.0,17.0,22705.0,95.0,2.0,2.0,2.0,86.0,2500.0,4.0,0.0,2.0,3.0,4541.0,2498.0,0.1,0.0,0.0,120.0,9.0,1.0,1.0,0.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,2.0,2.0,7.0,2.0,2.0,1.0,5.0,0.0,0.0,0.0,2.0,88.9,0.0,0.0,1.0,26501.0,22707.0,2500.0,24001.0,1282.0,635.0,639.0,Feb-2007,5.0,1.0,13.0,32.0,9.0,12.0,0.0,0.0,62.0,4,7,7,16,120.0,51000.0,122000.0,2007-02-01
375581,6000.0,6000.0,6000.0,36 months,20.39,224.18,4,19,PHLOBOTOMY,2,RENT,31000.0,0,Mar-2018,home_improvement,TX,20.79,0.0,705.0,709.0,0.0,NaN,NaN,9.0,0.0,0.0,0.0,9.0,w,0.0,NaN,1.0,2,114000.0,21.61,0,0.0,0.0,39627.0,0.0,7.0,1.0,3.0,8.0,39627.0,97.0,0.0,1.0,0.0,93.0,1500.0,1.0,0.0,3.0,4.0,4403.0,1000.0,0.0,0.0,0.0,38.0,70.0,19.0,8.0,0.0,19.0,NaN,8.0,NaN,0.0,0.0,0.0,1.0,1.0,7.0,2.0,2.0,0.0,9.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,42531.0,39627.0,1000.0,41031.0,5048.0,630.0,634.0,Aug-2006,3.0,0.0,9.0,82.8,4.0,5.0,0.0,0.0,NaN,4,8,8,19,70.0,24800.0,91200.0,2006-08-01
375584,7500.0,7500.0,7475.0,36 months,7.34,232.75,1,4,"Vice President, Client Services",10,OWN,300000.0,1,Mar-2018,other,CO,8.91,0.0,720.0,724.0,1.0,43.0,NaN,9.0,0.0,12750.0,34.6,23.0,f,0.0,68.0,1.0,2,500000.0,7.66,1,0.0,0.0,423667.0,2.0,2.0,1.0,1.0,6.0,54103.0,86.0,2.0,2.0,5143.0,57.0,36800.0,1.0,0.0,3.0,4.0,52958.0,23050.0,35.6,0.0,0.0,186.0,155.0,1.0,1.0,3.0,1.0,54.0,5.0,54.0,1.0,3.0,3.0,5.0,7.0,11.0,6.0,9.0,3.0,9.0,0.0,0.0,0.0,3.0,81.8,0.0,0.0,0.0,478415.0,66853.0,35800.0,73279.0,13308.0,600.0,604.0,Feb-2009,1.0,0.0,6.0,93.0,5.0,3.0,0.0,0.0,49.0,1,2,2,4,186.0,300000.0,500000.0,2009-02-01
375592,11000.0,11000.0,11000.0,60 months,17.47,276.17,4,16,Material Specialist,10,MORTGAGE,42000.0,0,Mar-2018,major_purchase,WA,19.00,3.0,685.0,689.0,0.0,14.0,76.0,8.0,1.0,5000.0,35.0,28.0,w,0.0,NaN,1.0,2,124000.0,6.44,0,0.0,0.0,270872.0,2.0,2.0,0.0,1.0,23.0,14380.0,50.0,2.0,2.0,3517.0,45.0,14300.0,1.0,1.0,2.0,3.0,38696.0,2118.0,69.7,0.0,0.0,137.0,215.0,4.0,4.0,3.0,4.0,NaN,0.0,NaN,0.0,2.0,3.0,2.0,4.0,11.0,5.0,14.0,3.0,8.0,

In [62]:
# creating a new column that will show the number of months that credit has existed for joint customer

acc_for_final['sec_app_duration_of_credit_months'] = (pd.DatetimeIndex(acc_for_final['issue_d']).year\
                                                      - pd.DatetimeIndex(acc_for_final['sec_app_earliest_cr_line']).year)*12 +\
(pd.DatetimeIndex(acc_for_final['issue_d']).month - pd.DatetimeIndex(acc_for_final['sec_app_earliest_cr_line']).month)

# converting null values to -1
acc_for_final.loc[acc_for_final['sec_app_duration_of_credit_months'].isnull(),
                  'sec_app_duration_of_credit_months'] = -1

acc_for_final['sec_app_duration_of_credit_months']

0         -1.0
1         -1.0
2         -1.0
3         -1.0
4         -1.0
          ... 
1348094   -1.0
1348095   -1.0
1348096   -1.0
1348097   -1.0
1348098   -1.0
Name: sec_app_duration_of_credit_months, Length: 1348099, dtype: float64

In [63]:
acc_for_final = acc_for_final.drop(['sec_app_earliest_cr_line_dt', 'sec_app_earliest_cr_line',
                                    'issue_d', 'sec_app_earliest_cr_line',
                                    'sec_app_earliest_cr_line_dt'], axis=1)

## Check the max values of columns starting with 'mths_since_'

In [64]:
cols_w_mths_since = [col for col in acc_for_final.columns if col.startswith('mths_since_')]
acc_for_final[cols_w_mths_since]

,mths_since_last_delinq,mths_since_last_record,mths_since_last_major_derog,mths_since_rcnt_il,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq
0,30.0,NaN,30.0,21.0,4.0,69.0,4.0,69.0
1,6.0,NaN,NaN,19.0,2.0,NaN,0.0,6.0
2,NaN,NaN,NaN,19.0,101.0,NaN,10.0,NaN
3,12.0,NaN,NaN,14.0,4.0,12.0,1.0,12.0
4,NaN,NaN,NaN,338.0,36.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1348094,NaN,NaN,NaN,15.0,17.0,NaN,2.0,NaN
1348095,NaN,NaN,NaN,8.0,70.0,NaN,3.0,NaN
1348096,NaN,NaN,NaN,16.0,4.0,NaN,15.0,NaN
1348097,68.0,26.0,68.0,4.0,17.0,NaN,12.0,NaN


In [65]:
acc_for_final[cols_w_mths_since].max()

mths_since_last_delinq            226.0
mths_since_last_record            129.0
mths_since_last_major_derog       226.0
mths_since_rcnt_il                511.0
mths_since_recent_bc              639.0
mths_since_recent_bc_dlq          202.0
mths_since_recent_inq              25.0
mths_since_recent_revol_delinq    202.0
dtype: float64

In [66]:
for col in cols_w_mths_since:
    acc_for_final.loc[acc_for_final[col].isnull(), col] = 10000

In [67]:
acc_for_final[cols_w_mths_since].max()

mths_since_last_delinq            10000.0
mths_since_last_record            10000.0
mths_since_last_major_derog       10000.0
mths_since_rcnt_il                10000.0
mths_since_recent_bc              10000.0
mths_since_recent_bc_dlq          10000.0
mths_since_recent_inq             10000.0
mths_since_recent_revol_delinq    10000.0
dtype: float64

## Get cols with null values

In [68]:
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_for_final, cols_w_null)

acc_for_final[cols_w_null]

,emp_title,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,collections_12_mths_ex_med,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,duration_of_credit_months,annual_inc_reduced,annual_inc_joint_reduced
0,leadman,55000.0,5.91,0.0,1.0,7.0,0.0,29.7,13.0,0.0,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.0,44000.0,NaN
1,Engineer,65000.0,16.06,1.0,4.0,22.0,0.0,19.2,38.0,0.0,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.0,52000.0,NaN
2,truck driver,63000.0,10.78,0.0,0.0,6.0,0.0,56.2,18.0,0.0,71000.0,13.85,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184.0,50400.0,56800.0
3,Contract Specialist,104433.0,25.37,1.0,3.0,12.0,0.0,64.5,35.0,0.0,NaN,NaN,0.0,0.0,331730.0,1.0,3.0,0.0,3.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210.0,104433.0,NaN
4,Veterinary Tecnician,34000.0,10.20,0.0,0.0,5.0,0.0,68.4,6.0,0.0,NaN,NaN,0.0,0.0,12798.0,0.0,1.0,0.0,0.0,3976.0,99.0,0.0,0.0,4522.0,76.0,12900.0,0.0,0.0,0.0,0.0,2560.0,844.0,91.0,0.0,0.0,338.0,54.0,32.0,32.0,0.0,0.0,2.0,3.0,2.0,2.0,2.0,4.0,4.0,3.0,5.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,16900.0,12798.0,9400.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,338.0,34000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348094,NaN,130000.0,20.59,0.0,1.0,17.0,0.0,34.0,39.0,0.0,NaN,NaN,0.0,0.0,168758.0,0.0,6.0,0.0,3.0,30516.0,52.0,0.0,3.0,7061.0,41.0,70100.0,1.0,2.0,1.0,6.0,9927.0,4161.0,68.2,0.0,0.0,113.0,146.0,17.0,15.0,3.0,0.0,3.0,9.0,3.0,5.0,21.0,10.0,15.0,9.0,17.0,0.0,0.0,0.0,0.0,100.0,33.3,0.0,0.0,275356.0,54349.0,13100.0,77756.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147.0,104000.0,NaN
1348095,Chief Operating Officer,180792.0,22.03,0.0,1.0,16.0,0.0,85.2,32.0,0.0,NaN,NaN,0.0,0.0,663465.0,0.0,4.0,2.0,4.0,70652.0,50.0,0.0,0.0

In [69]:
cols_w_null

['emp_title',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'annual_inc_joint',
 'dti_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec

## Fill in null values

In [70]:
# creates a list of the index where the column == 0

list(acc_for_final[acc_for_final['emp_length']==0].index)

[23,
 24,
 31,
 38,
 40,
 50,
 71,
 79,
 82,
 103,
 108,
 110,
 115,
 133,
 148,
 165,
 171,
 214,
 219,
 232,
 248,
 254,
 260,
 279,
 291,
 292,
 315,
 318,
 327,
 335,
 337,
 340,
 344,
 345,
 355,
 356,
 357,
 359,
 368,
 374,
 399,
 409,
 412,
 417,
 422,
 431,
 439,
 440,
 447,
 453,
 454,
 458,
 497,
 514,
 541,
 545,
 566,
 578,
 589,
 617,
 631,
 645,
 650,
 653,
 671,
 684,
 707,
 720,
 724,
 727,
 751,
 779,
 790,
 792,
 794,
 796,
 801,
 803,
 814,
 824,
 854,
 878,
 884,
 895,
 905,
 909,
 914,
 917,
 933,
 937,
 943,
 950,
 1015,
 1022,
 1041,
 1049,
 1066,
 1070,
 1081,
 1082,
 1093,
 1099,
 1100,
 1111,
 1138,
 1142,
 1143,
 1152,
 1154,
 1157,
 1160,
 1170,
 1177,
 1185,
 1187,
 1188,
 1194,
 1198,
 1200,
 1212,
 1220,
 1223,
 1225,
 1268,
 1281,
 1291,
 1292,
 1299,
 1332,
 1336,
 1361,
 1369,
 1420,
 1422,
 1426,
 1441,
 1465,
 1478,
 1479,
 1484,
 1485,
 1491,
 1494,
 1509,
 1524,
 1527,
 1536,
 1539,
 1540,
 1552,
 1555,
 1557,
 1569,
 1585,
 1588,
 1595,
 1603,
 1

In [71]:
# getting the columns where there are 0s

cols_w_0s_and_null = []

for col in cols_w_null:
    
    # if the list is not empty, then there are values == 0
    if list(acc_for_final[acc_for_final[col]==0].index)!=[]:
        cols_w_0s_and_null.append(col)
    
    for col_0 in cols_w_0s_and_null:
        acc_for_final.loc[acc_for_final[col_0].isnull(), col_0] = -1
    
acc_for_final

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,3,14,leadman,10,MORTGAGE,55000.0,0,debt_consolidation,PA,5.91,0.0,675.0,679.0,1.0,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,0.0,30.0,1.0,1,NaN,-1.00,-1,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,NaN,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,6,6,14,148.0,44000.0,NaN,-1.0
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,3,11,Engineer,10,MORTGAGE,65000.0,0,small_business,SD,16.06,1.0,715.0,719.0,4.0,6.0,10000.0,22.0,0.0,21470.0,19.2,38.0,w,0.0,10000.0,1.0,1,NaN,-1.00,-1,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,10000.0,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,-1.0,NaN,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,5,5,11,192.0,52000.0,NaN,-1.0
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,2,9,truck driver,10,MORTGAGE,63000.0,0,home_improvement,IL,10.78,0.0,695.0,699.0,0.0,10000.0,10000.0,6.0,0.0,7869.0,56.2,18.0,w,0.0,10000.0,1.0,2,71000.0,13.85,0,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,10000.0,10.0,10000.0,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,-1.0,NaN,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,4,4,9,184.0,50400.0,56800.0,-1.0
3,10400.0,10400.0,10400.0,60 months,22.45,289.91,6,26,Contract Specialist,3,MORTGAGE,104433.0,1,major_purchase,PA,25.37,1.0,695.0,699.0,3.0,12.0,10000.0,12.0,0.0,21929.0,64.5,35.0,w,0.0,10000.0,1.0,1,NaN,-1.00,-1,0.0,0.0,331730.0,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.

In [72]:
# rerun to get new cols w null values

cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_for_final, cols_w_null)

acc_for_final[cols_w_null]

,emp_title,total_acc,annual_inc_joint,mo_sin_old_rev_tl_op,sec_app_fico_range_low,sec_app_fico_range_high,duration_of_credit_months,annual_inc_joint_reduced
0,leadman,13.0,NaN,128.0,NaN,NaN,148.0,NaN
1,Engineer,38.0,NaN,192.0,NaN,NaN,192.0,NaN
2,truck driver,18.0,71000.0,184.0,NaN,NaN,184.0,56800.0
3,Contract Specialist,35.0,NaN,210.0,NaN,NaN,210.0,NaN
4,Veterinary Tecnician,6.0,NaN,54.0,NaN,NaN,338.0,NaN
...,...,...,...,...,...,...,...,...
1348094,NaN,39.0,NaN,146.0,NaN,NaN,147.0,NaN
1348095,Chief Operating Officer,32.0,NaN,174.0,NaN,NaN,175.0,NaN
1348096,Sales Manager,18.0,NaN,63.0,NaN,NaN,64.0,NaN
1348097,Manager,28.0,NaN,229.0,NaN,NaN,230.0,NaN


In [73]:
acc_for_final.loc[acc_for_final['emp_title'].isnull(), 'emp_title'] = 'Unknown'

### Employment length

In [74]:
emplength_mean = acc_for_final['emp_length'].mean()

In [75]:
acc_for_final.loc[(acc_for_final['emp_length'].isnull()) &\
                  (acc_for_final['emp_title'].notnull()), 'emp_length'] = emplength_mean

In [76]:
acc_for_final.loc[(acc_for_final['emp_length'].isnull()) &\
                  (acc_for_final['emp_title'].isnull()), 'emp_length'] = -1

### Employment Title

In [77]:
acc_for_final['emp_title'].value_counts()

Unknown                      85950
Teacher                      21268
Manager                      19472
Owner                        10302
Registered Nurse              8774
                             ...  
Tax Examnier                     1
Rakuten Linkshare                1
Maintenance  mechanic            1
Jewish Home & Care Center        1
Admin Support Supervisor         1
Name: emp_title, Length: 379864, dtype: int64

In [78]:
acc_for_final = acc_for_final.drop('emp_title', axis=1)

### joint

In [79]:
# if no secondary app, convert to 0

acc_for_final.loc[acc_for_final['annual_inc_joint'].isnull(), 'annual_inc_joint'] = -1

acc_for_final.loc[acc_for_final['annual_inc_joint_reduced'].isnull(),'annual_inc_joint_reduced'] = -1

acc_for_final.loc[acc_for_final['sec_app_fico_range_low'].isnull(), 'sec_app_fico_range_low'] = -1

acc_for_final.loc[acc_for_final['sec_app_fico_range_high'].isnull(), 'sec_app_fico_range_high'] = -1

### others

In [80]:
acc_for_final.loc[acc_for_final['annual_inc'].isnull(), 'annual_inc'] = -1

acc_for_final.loc[acc_for_final['mo_sin_old_rev_tl_op'].isnull(), 'mo_sin_old_rev_tl_op'] = 10000

acc_for_final.loc[acc_for_final['duration_of_credit_months'].isnull(),
                  'duration_of_credit_months'] = -1

acc_for_final.loc[acc_for_final['sec_app_duration_of_credit_months'].isnull(),
                  'sec_app_duration_of_credit_months'] = -1

acc_for_final.loc[acc_for_final['term']==60, 'term'] = 1
acc_for_final.loc[acc_for_final['term']==36, 'term'] = 0

### total accounts

In [81]:
cols_w_acc = [col for col in acc_for_final.columns if 'acc' in col]
acc_for_final[cols_w_acc]

,open_acc,total_acc,acc_now_delinq,open_acc_6m,acc_open_past_24mths,mo_sin_old_il_acct,mort_acc,num_accts_ever_120_pd,num_rev_accts,sec_app_mort_acc,sec_app_open_acc,sec_app_num_rev_accts
0,7.0,13.0,0.0,2.0,4.0,148.0,1.0,2.0,9.0,-1.0,-1.0,-1.0
1,22.0,38.0,0.0,1.0,4.0,113.0,4.0,0.0,27.0,-1.0,-1.0,-1.0
2,6.0,18.0,0.0,0.0,6.0,125.0,5.0,0.0,7.0,-1.0,-1.0,-1.0
3,12.0,35.0,0.0,1.0,10.0,128.0,6.0,0.0,19.0,-1.0,-1.0,-1.0
4,5.0,6.0,0.0,0.0,0.0,338.0,0.0,0.0,4.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1348094,17.0,39.0,0.0,0.0,6.0,113.0,3.0,0.0,15.0,-1.0,-1.0,-1.0
1348095,16.0,32.0,0.0,0.0,4.0,145.0,3.0,0.0,10.0,-1.0,-1.0,-1.0
1348096,14.0,18.0,0.0,2.0,10.0,53.0,3.0,0.0,11.0,-1.0,-1.0,-1.0
1348097,12.0,28.0,0.0,1.0,9.0,147.0,0.0,1.0,11.0,-1.0,-1.0,-1.0


In [82]:
# checking if we can use open accounts for null values

acc_for_final[(acc_for_final['total_acc'].isnull()) & (acc_for_final['open_acc'].isnull())][['total_acc', 'open_acc']]

,total_acc,open_acc


In [83]:
# using the open_acc as a filler for the null values for total_acc

acc_for_final.loc[acc_for_final['total_acc'].isnull(), 'total_acc'] = acc_for_final['open_acc']

In [84]:
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_for_final, cols_w_null)

acc_for_final[cols_w_null]

""
0
1
2
3
4
...
1348094
1348095
1348096
1348097


# Income : * Ratios

In [85]:
acc_for_final[(acc_for_final['application_type']==1) &\
              (acc_for_final['annual_inc_joint']==acc_for_final['annual_inc'])]\
[['annual_inc_joint', 'annual_inc']]

,annual_inc_joint,annual_inc
960645,-1.0,-1.0
960646,-1.0,-1.0
960676,-1.0,-1.0
960729,-1.0,-1.0


In [86]:
acc_for_final[acc_for_final['application_type']==2]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,2,9,10.0,MORTGAGE,63000.00,0,home_improvement,IL,10.78,0.0,695.0,699.0,0.0,10000.0,10000.0,6.0,0.0,7869.0,56.2,18.0,w,0.0,10000.0,1.0,2,71000.00,13.85,0,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,10000.0,10.0,10000.0,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,4,4,9,184.0,50400.00,56800.000,-1.0
388,15000.0,15000.0,15000.0,36 months,11.99,498.15,3,11,1.0,MORTGAGE,65000.00,1,debt_consolidation,CA,17.56,0.0,665.0,669.0,0.0,54.0,10000.0,9.0,0.0,13028.0,88.3,16.0,w,0.0,10000.0,1.0,2,77000.00,15.40,0,0.0,0.0,231860.0,0.0,0.0,0.0,0.0,30.0,22525.0,74.0,0.0,2.0,2839.0,78.0,14800.0,3.0,1.0,2.0,2.0,25762.0,295.0,94.2,0.0,0.0,99.0,136.0,13.0,13.0,1.0,28.0,10000.0,11.0,54.0,0.0,2.0,7.0,2.0,3.0,4.0,7.0,11.0,7.0,9.0,0.0,0.0,0.0,0.0,92.3,100.0,0.0,0.0,251645.0,35553.0,5100.0,30649.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,5,5,11,136.0,65000.00,61600.000,-1.0
451,24000.0,24000.0,24000.0,60 months,23.13,678.37,6,27,6.0,MORTGAGE,43435.00,1,debt_consolidation,TX,35.70,0.0,680.0,684.0,0.0,10000.0,84.0,13.0,1.0,12508.0,73.1,22.0,w,0.0,10000.0,1.0,2,113314.00,25.44,0,0.0,156.0,182215.0,0.0,6.0,2.0,5.0,10.0,48582.0,75.0,0.0,1.0,5690.0,74.0,17100.0,1.0,2.0,1.0,6.0,14017.0,357.0,96.8,0.0,0.0,52.0,209.0,13.0,10.0,1.0,26.0,10000.0,12.0,10000.0,0.0,2.0,5.0,2.0,4.0,9.0,6.0,12.0,5.0,13.0,0.0,0.0,0.0,2.0,100.0,100.0,1.0,0.0,208064.0,61090.0,11000.0,61739.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,11,11,22,209.0,43435.00,90651.200,-1.0
470,24000.0,24000.0,24000.0,60 months,13.44,551.50,3,13,1.0,MORTGAGE,96000.00,1,credit_card,MO,22.70,0.0,675.0,679.0,1.0,55.0,63.0,13.0,1.0,19403.0,53.3,24.0,w,0.0,55.0,1.0,2,117000.00,22.76,0,0.0,0.0,220879.0,0.0,4.0,1.0,3.0,10.0,43145.0,70.0,1.0,3.0,6210.0,64.0,36400.0,2.0,0.0,5.0,7.0,16991.0,6328.0,60.9,0.0,0.0,158.0,71.0,11.0,10.0,2.0,19.0,10000.0,6.0,55.0,1.0,2.0,5.0,3.0,4.0,8.0,8.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,95.7,66.7,0.0,0.0,258301.0,625

In [87]:
acc_for_final[acc_for_final['annual_inc_joint']>0]['annual_inc_joint']

2           71000.00
388         77000.00
451        113314.00
470        117000.00
581         94348.32
             ...    
1347958     48828.00
1348054    105000.00
1348056    106500.00
1348062    154000.00
1348082    103000.00
Name: annual_inc_joint, Length: 25806, dtype: float64

In [88]:
acc_for_final['annual_inc_joint'].isnull()

0          False
1          False
2          False
3          False
4          False
           ...  
1348094    False
1348095    False
1348096    False
1348097    False
1348098    False
Name: annual_inc_joint, Length: 1348099, dtype: bool

In [89]:
# Income to loan amount ratio

acc_for_final['ratio_inc_loan'] = acc_for_final['annual_inc'] / acc_for_final['loan_amnt']


# income reduced to loan amount ratio

acc_for_final['ratio_redinc_loan'] = acc_for_final['annual_inc_reduced'] / acc_for_final['loan_amnt']

# Income to funded amount ratio

acc_for_final['ratio_inc_funded'] = acc_for_final['annual_inc'] / acc_for_final['funded_amnt']


# income reduced to funded amount ratio

acc_for_final['ratio_redinc_funded'] = acc_for_final['annual_inc_reduced'] /\
acc_for_final['funded_amnt']

In [90]:
# Income to loan amount ratio

acc_for_final['ratio_incjoint_loan'] = acc_for_final['annual_inc_joint'] / acc_for_final['loan_amnt']


# income reduced to loan amount ratio

acc_for_final['ratio_redincjoint_loan'] = acc_for_final['annual_inc_joint_reduced'] /\
acc_for_final['loan_amnt']

# Income to funded amount ratio

acc_for_final['ratio_incjoint_funded'] = acc_for_final['annual_inc_joint'] /\
acc_for_final['funded_amnt']


# income reduced to funded amount ratio

acc_for_final['ratio_redincjoint_funded'] = acc_for_final['annual_inc_joint_reduced'] /\
acc_for_final['funded_amnt']

# Loan Amount vs Funded Amount

In [91]:
acc_for_final['loan_minus_funded'] = acc_for_final['loan_amnt'] - acc_for_final['funded_amnt']

acc_for_final['ratio_loan_funded'] = acc_for_final['loan_amnt'] / acc_for_final['funded_amnt']

## Downsampling the dataset

In [92]:
from sklearn.utils import resample

label = target_final['loan_status_binary']

target_mean = label.mean()

target_fullypaid = label == 0

subsample_ind = resample(label[target_fullypaid].index.values, \
                         replace=False, n_samples=int(1348099*target_mean))

combined = np.concatenate([subsample_ind, label[~target_fullypaid].index.values])

acc_subsample = acc_for_final.iloc[combined]
target_subsample = target_final[['loan_status_binary']].iloc[combined]

acc_subsample = acc_subsample.reset_index()
acc_subsample = acc_subsample.drop('index', axis=1)

target_subsample = target_subsample.reset_index()
target_subsample = target_subsample.drop('index', axis=1)

target_subsample.mean()

loan_status_binary    0.5
dtype: float64

In [93]:
acc_subsample

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,ratio_inc_loan,ratio_redinc_loan,ratio_inc_funded,ratio_redinc_funded,ratio_incjoint_loan,ratio_redincjoint_loan,ratio_incjoint_funded,ratio_redincjoint_funded,loan_minus_funded,ratio_loan_funded
0,20000.0,12200.0,10098.117743,36 months,6.17,372.09,1,3,3.0,RENT,75000.0,1,debt_consolidation,MA,8.42,0.0,775.0,779.0,3.0,10000.0,10000.0,10.0,0.0,9608.0,27.2,23.0,f,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,10000.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,10000.0,-1.0,-1.0,-1.0,10000.0,10000.0,10000.0,10000.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,2,1,3,309.0,75000.0,-1.0,-1.0,3.750000,3.750000,6.147541,6.147541,-0.000050,-0.000050,-0.000082,-0.000082,7800.0,1.639344
1,15000.0,15000.0,15000.000000,36 months,9.17,478.19,2,7,10.0,RENT,66000.0,1,debt_consolidation,CA,17.27,1.0,680.0,684.0,0.0,9.0,10000.0,8.0,0.0,15668.0,67.8,22.0,w,0.0,52.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,30695.0,-1.0,-1.0,-1.0,-1.0,10000.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,23100.0,-1.0,-1.0,-1.0,3.0,3837.0,5321.0,53.7,0.0,0.0,100.0,194.0,11.0,11.0,3.0,11.0,9.0,10000.0,9.0,1.0,3.0,5.0,4.0,5.0,4.0,7.0,15.0,5.0,8.0,0.0,0.0,0.0,1.0,90.9,25.0,0.0,0.0,53450.0,30695.0,11500.0,30350.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,3,3,7,194.0,66000.0,-1.0,-1.0,4.400000,4.400000,4.400000,4.400000,-0.000067,-0.000067,-0.000067,-0.000067,0.0,1.000000
2,35000.0,35000.0,35000.000000,60 months,22.40,974.64,5,23,2.0,RENT,142500.0,1,credit_card,CA,24.80,0.0,710.0,714.0,3.0,10000.0,10000.0,17.0,0.0,21739.0,66.7,27.0,f,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,202230.0,-1.0,-1.0,-1.0,-1.0,10000.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,33000.0,-1.0,-1.0,-1.0,8.0,11896.0,3001.0,83.9,0.0,0.0,133.0,323.0,18.0,2.0,0.0,18.0,10000.0,4.0,10000.0,0.0,6.0,9.0,7.0,11.0,9.0,11.0,18.0,9.0,17.0,0.0,0.0,0.0,4.0,100.0,57.1,0.0,0.0,243198.0,202230.0,15400.0,210198.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,10,9,23,324.0,142500.0,-1.0,-1.0,4.071429,4.071429,4.071429,4.071429,-0.000029,-0.000029,-0.000029,-0.000029,0.0,1.000000
3,13200.0,13

In [94]:
target_subsample

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
538715,1
538716,1
538717,1
538718,1


In [95]:
target_subsample.value_counts()

loan_status_binary
0                     269360
1                     269360
dtype: int64

# Dummification

In [96]:
# getting a list of cols w dtype object

cols_w_obj = list(acc_for_final.select_dtypes(include=['object']).columns)

cols_w_obj

['term', 'home_ownership', 'purpose', 'addr_state', 'initial_list_status']

In [97]:
# dummification

acc_dum = pd.get_dummies(acc_subsample, drop_first=True)

acc_dum

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,ratio_inc_loan,ratio_redinc_loan,ratio_inc_funded,ratio_redinc_funded,ratio_incjoint_loan,ratio_redincjoint_loan,ratio_incjoint_funded,ratio_redincjoint_funded,loan_minus_funded,ratio_loan_funded,term_ 60 months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w
0,20000.0,12200.0,10098.117743,6.17,372.09,1,3,3.0,75000.0,1,8.42,0.0,775.0,779.0,3.0,10000.0,10000.0,10.0,0.0,9608.0,27.2,23.0,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,10000.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,10000.0,-1.0,-1.0,-1.0,10000.0,10000.0,10000.0,10000.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,2,1,3,309.0,75000.0,-1.0,-1.0,3.750000,3.750000,6.147541,6.147541,-0.000050,-0.000050,-0.000082,-0.000082,7800.0,1.639344,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15000.0,15000.0,15000.000000,9.17,478.19,2,7,10.0,66000.0,1,17.27,1.0,680.0,684.0,0.0,9.0,10000.0,8.0,0.0,15668.0,67.8,22.0,0.0,52.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,30695.0,-1.0,-1.0,-1.0,-1.0,10000.0,-1.0,

## Splitting up the downsampled dataframes into halves so that we have some data for predictions

In [98]:
# getting indices of half of the samples

target_subsample_index_0 = sorted(list(target_subsample[target_subsample['loan_status_binary']==0].sample(frac=0.5, random_state=0).index))
target_subsample_index_1 = sorted(list(target_subsample[target_subsample['loan_status_binary']==1].sample(frac=0.5, random_state=0).index))

target_subsample_index_0and1 = target_subsample_index_0 + target_subsample_index_1
target_subsample_index_0and1

[0,
 5,
 8,
 12,
 13,
 14,
 16,
 17,
 18,
 19,
 20,
 23,
 24,
 25,
 28,
 29,
 31,
 32,
 34,
 36,
 39,
 41,
 44,
 45,
 47,
 48,
 49,
 51,
 52,
 56,
 57,
 61,
 62,
 63,
 65,
 67,
 68,
 69,
 70,
 75,
 77,
 78,
 81,
 88,
 89,
 96,
 97,
 98,
 100,
 103,
 104,
 105,
 116,
 117,
 119,
 121,
 122,
 124,
 125,
 127,
 128,
 133,
 136,
 137,
 139,
 141,
 143,
 144,
 145,
 149,
 151,
 152,
 156,
 157,
 159,
 160,
 161,
 162,
 165,
 166,
 167,
 168,
 170,
 177,
 179,
 180,
 181,
 184,
 185,
 187,
 190,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 204,
 206,
 207,
 208,
 209,
 210,
 212,
 213,
 214,
 215,
 216,
 220,
 222,
 225,
 226,
 230,
 234,
 236,
 237,
 240,
 242,
 243,
 248,
 250,
 252,
 256,
 258,
 259,
 264,
 265,
 266,
 267,
 270,
 271,
 272,
 273,
 274,
 275,
 278,
 281,
 282,
 284,
 285,
 286,
 287,
 290,
 292,
 295,
 296,
 297,
 301,
 302,
 303,
 305,
 306,
 309,
 314,
 316,
 319,
 320,
 321,
 329,
 331,
 337,
 338,
 339,
 340,
 342,
 345,
 346,
 350,
 351,
 354,
 355,
 356,
 357,
 

In [99]:
# getting the other half of the indices

target_pred_index = sorted(list(set(list(range(0,
                                               target_subsample['loan_status_binary'].count()))) -\
                                set(target_subsample_index_0and1)))

target_pred_index

[1,
 2,
 3,
 4,
 6,
 7,
 9,
 10,
 11,
 15,
 21,
 22,
 26,
 27,
 30,
 33,
 35,
 37,
 38,
 40,
 42,
 43,
 46,
 50,
 53,
 54,
 55,
 58,
 59,
 60,
 64,
 66,
 71,
 72,
 73,
 74,
 76,
 79,
 80,
 82,
 83,
 84,
 85,
 86,
 87,
 90,
 91,
 92,
 93,
 94,
 95,
 99,
 101,
 102,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 118,
 120,
 123,
 126,
 129,
 130,
 131,
 132,
 134,
 135,
 138,
 140,
 142,
 146,
 147,
 148,
 150,
 153,
 154,
 155,
 158,
 163,
 164,
 169,
 171,
 172,
 173,
 174,
 175,
 176,
 178,
 182,
 183,
 186,
 188,
 189,
 191,
 192,
 201,
 202,
 203,
 205,
 211,
 217,
 218,
 219,
 221,
 223,
 224,
 227,
 228,
 229,
 231,
 232,
 233,
 235,
 238,
 239,
 241,
 244,
 245,
 246,
 247,
 249,
 251,
 253,
 254,
 255,
 257,
 260,
 261,
 262,
 263,
 268,
 269,
 276,
 277,
 279,
 280,
 283,
 288,
 289,
 291,
 293,
 294,
 298,
 299,
 300,
 304,
 307,
 308,
 310,
 311,
 312,
 313,
 315,
 317,
 318,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 330,
 332,
 333,
 334,
 335,
 336,
 341,
 34

In [100]:
# removing 1/2 of the dataset to use to compare our predictive model

# creating dataframes using 1/2 of the indices
acc_nonorm_touse = acc_dum.iloc[target_subsample_index_0and1]
target_touse = target_subsample.iloc[target_subsample_index_0and1]


# creating dataframes using the other 1/2 of the indices
acc_nonorm_forprediction = acc_dum.iloc[target_pred_index]
target_forprediction = target_subsample.iloc[target_pred_index]

In [101]:
acc_nonorm_touse.to_csv("acc_nonorm_touse.csv.gz",
                        index=False,
                        compression="gzip")

acc_nonorm_forprediction.to_csv("acc_nonorm_forprediction.csv.gz",
                                index=False,
                                compression="gzip")

target_touse.to_csv("target_touse.csv.gz",
                           index=False,
                           compression="gzip")

target_forprediction.to_csv("target_forprediction.csv.gz",
                                   index=False,
                                   compression="gzip")

# Scaling the Dataset

In [102]:
# creating copies

import copy

acc_partial = copy.deepcopy(acc_dum)
acc_full = copy.deepcopy(acc_dum)

## partial scaling

In [103]:
# partial scaling

from sklearn.preprocessing import MinMaxScaler

acc_dum_cols = list(acc_dum.columns)

for col in acc_dum_cols:
    if acc_partial[col].max() >= 100:
        acc_partial[col] = MinMaxScaler().fit_transform(np.array(acc_partial[col]).reshape(-1,1))

In [104]:
# separating the partially normalized dataframe into 2 dataframes for use and prediction comparison

acc_partialnorm_touse = acc_partial.iloc[target_subsample_index_0and1]

acc_partialnorm_forprediction = acc_partial.iloc[target_pred_index]

In [105]:
acc_partialnorm_touse.to_csv("acc_partialnorm_touse.csv.gz",
                             index=False,
                             compression="gzip")

acc_partialnorm_forprediction.to_csv("acc_partialnorm_forprediction.csv.gz",
                                     index=False,
                                     compression="gzip")

## full scaling

In [106]:
# full scaling

for col in acc_dum_cols:
    acc_full[col] = MinMaxScaler().fit_transform(np.array(acc_full[col]).reshape(-1,1))

In [107]:
# separating the fully normalized dataframe into 2 dataframes for use and prediction comparison

acc_fullnorm_touse = acc_full.iloc[target_subsample_index_0and1]

acc_fullnorm_forprediction = acc_full.iloc[target_pred_index]

In [108]:
acc_fullnorm_touse.to_csv("acc_fullnorm_touse.csv.gz",
                             index=False,
                             compression="gzip")

acc_fullnorm_forprediction.to_csv("acc_fullnorm_forprediction.csv.gz",
                                     index=False,
                                     compression="gzip")